In [1]:
%matplotlib inline

import mido
import numpy as np
from math import ceil
import keras
import random
import music21
import glob
import os

Using TensorFlow backend.


In [2]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

In [3]:

#Args: mido.mido_obj
# output notelist


In [4]:
twin = mido.MidiFile('data/tr_guitar_licks/C_Lick1.mid')

In [5]:
def get_notelist(mido_obj, res=8, note_range=(36,84)):
    """
    Args: 
        res (int) - resolution.  8 is eighth notes, 4 quarter, etc...
        note_range (tuple) - middle C = 60
    """

    # this gives # of ticks in a column of the piano sheet
    tick_step = round(float(mido_obj.ticks_per_beat/res))

    active_notes = [0] * abs(np.diff(note_range)[0])

    counter = 0
    notelist = []

    rawmsgs = [msg for track in mido_obj.tracks for msg in track]

    # have to reset counter when a new track is detected
    for msg in rawmsgs:
        if msg.type not in set(['note_on','note_off']):
            continue

        #counter += ceil(msg.time/tick_step)
        counter += msg.time

        if msg.type == 'note_on' and msg.velocity > 0:        
            active_notes[msg.note - note_range[0]] = counter

        elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
            # fill everything up to this with 1s
            start = active_notes[msg.note - note_range[0]]  #round this section

            notelist.append((msg.note, int(ceil(start/tick_step)), int(ceil((counter-start)/tick_step))))
            active_notes[msg.note - note_range[0]] = 0

    notelist = sorted(notelist, key=lambda x: x[1])
    return (counter, notelist)

In [6]:
def make_musicmat(notelist, steps, note_range=(36,84)):
    music_mat = np.zeros([steps, int(np.diff(note_range))])
    cnt = 0
    for nt in notelist:
        c = range(nt[1], (nt[2] + nt[1]))
        r = [nt[0] - note_range[0]] * len(c)
        music_mat[c,r] = np.array([1] * len(r) )
        cnt += 1
    return music_mat

In [7]:
def segmentize(data, maxlen, step):
    X = []
    y = []
    steps = data.shape[0]
    for ix in range(0, steps-maxlen-1, step):
        end = ix + maxlen
        X.append(data[ix:end:1, :])
        y.append(data[end+1,:])
    X = np.stack(X)
    y = np.stack(y)
    return X,y

In [8]:
#params

res = 8
note_range=(36,84)

In [9]:
raw_data = []

guitar_dir = 'data/tr_guitar_licks/'
for fname in glob.glob(guitar_dir + '*'):
    try:
        midifile = mido.MidiFile(fname)
        counter, notelist = get_notelist(midifile)
        tick_step = round(float(midifile.ticks_per_beat/res))
        steps = int(ceil(counter/tick_step))
        music_mat = make_musicmat(notelist, steps)
        raw_data.append(music_mat)
    except:
        pass

### Machine readable inputs

In [10]:
maxlen = 50

X = []
y = []
for sample in raw_data:
    smpX, smpy = segmentize(sample, maxlen, 1)
    X.append(smpX)
    y.append(smpy)

X = np.concatenate(X, axis=0)
y = np.concatenate(y, axis=0)

### Train the model

In [11]:
nnotes = np.diff(note_range)[0]
maxlen = 50
step = 1

In [12]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, nnotes)))
model.add(Dense(nnotes))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [15]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, nnotes)))
model.add(Dense(nnotes))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(trainX, trainY, nb_epoch=100, batch_size=1, verbose=2)

In [16]:
#X = X.astype(bool)
model.fit(X,y, batch_size=50, nb_epoch=1)

Epoch 1/1


InternalError: Blas SGEMM launch failed : a.shape=(2500, 48), b.shape=(48, 128), m=2500, n=128, k=48
	 [[Node: MatMul_15 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](Reshape_24, lstm_4_W_i/read)]]
	 [[Node: mul_9/_27 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1711_mul_9", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'MatMul_15', defined at:
  File "/home/alex/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/alex/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/alex/.local/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/alex/.local/lib/python2.7/site-pa

In [14]:
y.shape

(3733, 48)

In [15]:
X.shape

(3733, 50, 48)